In [1]:
# MARK:- libs
import pandas as pd 
import numpy as np 
from datetime import datetime, date, timedelta

In [2]:
# MARK:- read input
path = 'reservation.csv'
data_df = pd.read_csv(path)
data_df.check_in_date = pd.to_datetime(data_df.check_in_date)
data_df.check_out_date = pd.to_datetime(data_df.check_out_date)

In [3]:
# MARK:- calculating number of nights and price for each night for each period
data_df['night_no'] = data_df.apply(lambda row: (row['check_out_date'] - row['check_in_date']).days, axis=1)
data_df['punit'] = data_df.price / data_df.night_no

In [5]:
# MARK:- requirements for function f
min_range = min(data_df.check_in_date).date()
max_range = max(data_df.check_out_date).date()
relieve = pd.DataFrame(pd.date_range(min_range, max_range))
relieve.columns = ['day']
relieve['available'] = relieve.day.apply(lambda x: [row['punit'] for idx, row in data_df.iterrows() if row['check_in_date'] <= x < row['check_out_date']])

In [6]:
def f(x, hops=10):
    if x not in relieve.day.astype(str).values:
        return (0,0)
    booking = relieve[relieve.day.astype(str) == x].available.values[0]
    booking = sorted(booking)
    if len(booking) == 0:
        return (0,0)
    if len(booking) == 1:
        return (booking[0], booking[0])
    y = 0
    max_profit = 0
    idx = 0
    for idx in range(len(booking)):
        threshold = booking[idx]
        profit = threshold * len([b for b in booking if b >= threshold])
        if profit >= max_profit:
            max_profit = profit
            update = True
            y = threshold
        
    return (y, max_profit)

In [7]:
profit = relieve.day.astype(str).apply(lambda x: f(x)[1]).values
max_sum = sum(profit)
max_sum

198958254.0